### Goal of this notebook
Use iris dataset
- one hot encode it
- pipeline it
- cross validation
- xgbregressor model

In [1]:
import pandas as pd

In [2]:
iris = pd.read_csv('Iris.csv')

In [3]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [26]:
y = iris['Species']
X = iris.drop(['Species', 'Id'], axis=1)

In [27]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=256)

### One Hot Encoding
one hot encode the y data

In [52]:
from sklearn.preprocessing import OneHotEncoder

encode = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [54]:
reshaped_y_train = y_train.values.reshape(-1,1) # use .values.reshape since "series object has no attribute reshape"
reshaped_y_test = y_test.values.reshape(-1,1)

In [55]:
cat_y_train = pd.DataFrame(encode.fit_transform(reshaped_y_train))
cat_y_test = pd.DataFrame(encode.transform(reshaped_y_test))

In [56]:
# Make this into one function
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def check_model(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mae = mean_absolute_error(predictions, y_test)
    print(mae)

In [57]:
check_model(X_train, X_test, cat_y_train, cat_y_test)

0.024736842105263158


## XGBoost

In [67]:
reshaped_y = y.values.reshape(-1,1)
y = pd.DataFrame(encode.fit_transform(reshaped_y))

In [68]:
from xgboost import XGBRegressor
modelXGB = XGBRegressor(n_estimators=500, learn_rate=0.05)
modelXGB.fit(X, y, early_stopping_rounds=5, eval_set=[(X_test, cat_y_test)], verbose=False)

AssertionError: 

## Pipelines

In [46]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# preprocessing y data
label_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))
                                   ])

preprocessor = ColumnTransformer(
    transformers=[
        ('label', label_transformer, y)
    ]
)

model = RandomForestRegressor(n_estimators=100, random_state=256)

# Bundle preprocessing and modeling code

bundle = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

bundle.fit(X_train, y_train)

predictions = bundle.predict(X_test)
mae = mean_absolute_error(predictions, y_test)
mae

ValueError: A given column is not a column of the dataframe